<a href="https://colab.research.google.com/github/takacsanna/igy_neveld_fakt/blob/main/ora6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##6. óra

Innentől csak a hálózatok és rétegek kombinációival fogunk foglalkozni

sima fully connected: nem képes a szekvenciális (az adatpontok egymáshoz mért helye információt jelent) adatok kezelésére, ÉS mindig csak az adott inputot veszi figyelembe, de a súlyokon kívül nincs memóriája (pl. a korábbi elemekről)

--> a konvolúciós hálózat valamennyire megoldást nyújt rá, de van szofisztikáltabb megoldás is

---> **REKURRENS HÁLÓ**
-- memóriát viszünk a modellbe

--> rajz

dia: RNN

rövidtávú memóriája vana a hálózatnak --> mindig a felét becsüli az aktuális bemenettel, mindig feleződik a korábbi



LSTM háló (Long Short Term Memory)

ugyanúgy réteg --> valamivel bonyolultabb; kapukkal módsítjuk a továbbhaladó információt

két sáv: fent: halad a hosszútávú

lent: konkatenált mátrixok haladnak tovább; szigmoid függvénnyel átformáljuk, tanH-val... mindegy, ábra

felhasználás: természetes nyelvfeldoglozás (NLP) -- chatbot, tartalomkinyerés, szöveggenerálás, sentiment analízis; képleírás; video to text; idősorok; többféle signal feldolgozása

In [ ]:
#gutenberg.org

##Szövegformázás

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
raw_text=open('/content/drive/My Drive/Python kurzus/22690.txt', 'r').read()

In [ ]:
raw_text

"\x0c\x0c\x0c\x0c\x0cMAGYAR KÖNYVTÁR.\n\nBERNARD SHAW\n\nCAESAR ÉS CLEOPATRA\nEGY DARAB TÖRTÉNELEM\nÖT FELVONÁSBAN\nFordította\n\nMIKES LAJOS\n\nBUDAPEST\n\nLAMPELR.kk. (WodIanfr F. és Fiai) r. t.\nKÖNYVKIADÓVALLALATA\n\n\x0cMAGYAR KÖNYVTÁR\nSzerkeszti RADÓ ANTAL.\nElsőrangú hazai és külföldi Írók regényei; elbeszélések, versek,\nszínmüvek, ismeretterjesztő müvek sorozata.\n\nMinden füzet külön is kapható.\n\n*\n\n497. Ady Endre. Sápadt em\nberek és történetek.\n498 -499. Wilde Oszkár. De\nProfundis. Ford. Mikes Lajos.\n500 Beöthy Zsolt. Költők es\nhősök.\n501. Krúdy Gy. Hét szilvafa.\nElbeszélések.\n502. Wilde Oszkár Aforiz\xad\nmák és ötletek. Ford. Radó\n\n518. Gustave\n\nTeréz.\n\n508. Alphonse Daudet. Muvészházassógok.\nFordította\n\nEgy-egy szám ára 30 fillér\n\njó\n\nlélek.\n\nFlaubert. Eg\n\nFordította\n\nGábu\n\nAndor.\n519. Shakespere. Juli s Cae\n\nsár. Fordította Vörösmarí\nMihály.\n5.0. Farkas Fái. Keleti U1\nképek.\n21\n\n522.\n\nHalász\n\nZoltár\n\nMester-játszmák.\n523 

In [ ]:
#Kisbetűkké változtatás

raw_text=raw_text.lower()

In [ ]:
raw_text

"\x0c\x0c\x0c\x0c\x0cmagyar könyvtár.\n\nbernard shaw\n\ncaesar és cleopatra\negy darab történelem\nöt felvonásban\nfordította\n\nmikes lajos\n\nbudapest\n\nlampelr.kk. (wodianfr f. és fiai) r. t.\nkönyvkiadóvallalata\n\n\x0cmagyar könyvtár\nszerkeszti radó antal.\nelsőrangú hazai és külföldi írók regényei; elbeszélések, versek,\nszínmüvek, ismeretterjesztő müvek sorozata.\n\nminden füzet külön is kapható.\n\n*\n\n497. ady endre. sápadt em\nberek és történetek.\n498 -499. wilde oszkár. de\nprofundis. ford. mikes lajos.\n500 beöthy zsolt. költők es\nhősök.\n501. krúdy gy. hét szilvafa.\nelbeszélések.\n502. wilde oszkár aforiz\xad\nmák és ötletek. ford. radó\n\n518. gustave\n\nteréz.\n\n508. alphonse daudet. muvészházassógok.\nfordította\n\negy-egy szám ára 30 fillér\n\njó\n\nlélek.\n\nflaubert. eg\n\nfordította\n\ngábu\n\nandor.\n519. shakespere. juli s cae\n\nsár. fordította vörösmarí\nmihály.\n5.0. farkas fái. keleti u1\nképek.\n21\n\n522.\n\nhalász\n\nzoltár\n\nmester-játszmák.\n523 

In [ ]:
#Csúnya karakterek törlése
raw_text=raw_text.replace('\n', ' ')
raw_text=raw_text.replace('\t', ' ')
raw_text=raw_text.replace('\x0c', ' ')

In [ ]:
import re

In [ ]:
raw_text = re.sub(r'[_"\-;%()|,+&/=*%:#@\[\]]', ' ', raw_text)

In [ ]:
raw_text

"     magyar könyvtár.  bernard shaw  caesar és cleopatra egy darab történelem öt felvonásban fordította  mikes lajos  budapest  lampelr.kk.  wodianfr f. és fiai  r. t. könyvkiadóvallalata   magyar könyvtár szerkeszti radó antal. elsőrangú hazai és külföldi írók regényei  elbeszélések  versek  színmüvek  ismeretterjesztő müvek sorozata.  minden füzet külön is kapható.     497. ady endre. sápadt em berek és történetek. 498  499. wilde oszkár. de profundis. ford. mikes lajos. 500 beöthy zsolt. költők es hősök. 501. krúdy gy. hét szilvafa. elbeszélések. 502. wilde oszkár aforiz\xad mák és ötletek. ford. radó  518. gustave  teréz.  508. alphonse daudet. muvészházassógok. fordította  egy egy szám ára 30 fillér  jó  lélek.  flaubert. eg  fordította  gábu  andor. 519. shakespere. juli s cae  sár. fordította vörösmarí mihály. 5.0. farkas fái. keleti u1 képek. 21  522.  halász  zoltár  mester játszmák. 523 conan doyle. sherloe ho mes k i andjai. f fayor rózsa.  l'orc  524—525. schopenhauer a1 e

In [ ]:
#A betűket számmá alakítjuk

random_v = [1,2,45,3,3,3,5,5,5]

In [ ]:
set(random_v)

{1, 2, 3, 5, 45}

In [ ]:
chars = sorted(list(set(raw_text)))

In [ ]:
chars[:10]

[' ', '!', "'", '.', '0', '1', '2', '3', '4', '5']

In [ ]:
char_to_int = {}
int_to_char = {}

for i, char in enumerate(chars):
    char_to_int[char] = i
    int_to_char[i] = char


In [ ]:
#int_to_char

In [ ]:
n_chars=len(raw_text)
n_vocab=len(chars)
print('A szöveg teljes karaktertartalma: ', n_chars)
print('Ennyi karakter van benne: ', n_vocab)

A szöveg teljes karaktertartalma:  206789
Ennyi karakter van benne:  68


In [ ]:
#feldaraboljuk a szöveget szekvenciákra

maxlen=60
step=3
sentences=[]
next_chars=[]

for i in range (0, len(raw_text) - maxlen, step):
  sentences.append(raw_text[i:i+maxlen])
  next_chars.append(raw_text[i+maxlen])

In [ ]:
sentences[0]

'     magyar könyvtár.  bernard shaw  caesar és cleopatra egy'

In [ ]:
next_chars[0]

' '

In [ ]:
len(next_chars)

68910

In [ ]:
#One-hot encoding --> eddig paranccsal csiináltuk, most kézzel:(

import numpy as np

x_train = np.zeros((len(sentences), maxlen, n_vocab))

In [ ]:
x_train.shape

(68910, 60, 68)

In [ ]:
y_train = np.zeros((len(sentences), n_vocab))

In [ ]:
for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x_train[i, t, char_to_int[char]]=1
    y_train[i, char_to_int[next_chars[i]]]=1

##Rekurrens háló

In [ ]:
from tensorflow.keras import layers
from tensorflow import keras

In [ ]:
#osztályozási feladat, ezért softmax --> dense réteggel végezzük, amiben annyi neuron van, ahány egyéni betű
input=layers.Input(shape=(maxlen, n_vocab))
x=layers.LSTM(256)(input)
x=layers.Dropout(0.2)(x)
output=layers.Dense(n_vocab, activation='softmax')(x)

In [ ]:
model=keras.Model(input, output)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
#model.fit(x_train, y_train)

##Hogyan teljesít tanulás közben? 

In [ ]:
import random

In [ ]:
starting_point=random.randint(0, len(raw_text)-maxlen-1)
sentence = raw_text[starting_point : starting_point+maxlen]

In [ ]:
sentence

'on azt fogja parancsolni neki  hogy öljön meg téged is. már '

In [ ]:
x_pred = np.zeros((1, maxlen, n_vocab))
for t, char in enumerate(sentence):
    x_pred[0, t, char_to_int[char]] = 1.0

In [ ]:
x_pred.shape

(1, 60, 68)

In [ ]:
model.predict(x_pred)

array([[0.01440371, 0.0145233 , 0.0145521 , 0.01465358, 0.01494104,
        0.01485666, 0.01505763, 0.0142583 , 0.01502992, 0.01522579,
        0.01453705, 0.01493006, 0.01431099, 0.01537911, 0.01450732,
        0.01424736, 0.01429073, 0.01516671, 0.01471484, 0.01462683,
        0.0146651 , 0.01531174, 0.0148801 , 0.01436519, 0.01469104,
        0.01475056, 0.01518513, 0.01443944, 0.01509145, 0.01479148,
        0.01457094, 0.01510547, 0.01484288, 0.01424373, 0.01493882,
        0.01516833, 0.01447679, 0.01489145, 0.01452241, 0.01438347,
        0.01519452, 0.01493005, 0.01450702, 0.01430047, 0.01525881,
        0.01486268, 0.01427491, 0.01483896, 0.01458506, 0.01479695,
        0.01427575, 0.0143396 , 0.01463722, 0.01501078, 0.01433307,
        0.01501184, 0.01410151, 0.0140188 , 0.01507834, 0.01441069,
        0.01461384, 0.01445448, 0.01505128, 0.01446175, 0.01466322,
        0.01486101, 0.01480161, 0.01479719]], dtype=float32)

In [ ]:
np.argmax(model.predict(x_pred))

13

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

##Tanítás + kiértékelés

In [ ]:
epochs = 10

for epoch in range(epochs):
    print('------ Epoch ', epoch)

    model.fit(x_train, y_train, epochs=1, batch_size=128)

    starting_point = random.randint(0, len(raw_text) - maxlen - 1)
    for temp in [0.1, 1.0]:
        generated = ''
        sentence = raw_text[starting_point : starting_point + maxlen]
        print('A kiindulási mondatunk: ', sentence)

        for _ in range(200):
            # Átformázzuk erre az alakra: [samples, time steps, features]
            x_pred = np.zeros((1, maxlen, n_vocab))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_to_int[char]] = 1.0

            preds = model.predict(x_pred)[0]
            next_index = sample(preds, temp) # np.argmax()
            next_char = int_to_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char
        print(f'A generált szöveg {temp} temperature-rel: {generated}')
    print()

------ Epoch  0
539/539 [==============================] - 6s 11ms/step - loss: 1.5928
A kiindulási mondatunk:  ba ki roppant mód gyönyörködik brit titkárának üzletszerű er
A generált szöveg 0.1 temperature-rel: ősztelen királynő vannak  hogy meg az ellenek a tengerbe  mert a ki közények a karonában  mint a teszért középen  a mint mert meg az egyiptomiak  hogy meg a leszem  a mint mert meg a leszélő  a mint m
A kiindulási mondatunk:  ba ki roppant mód gyönyörködik brit titkárának üzletszerű er
A generált szöveg 1.0 temperature-rel: szöggenged felő a lholombab  bíik térten is és kosszút rűnénym att a delem. caesar. a telsz.  miat neszt   hadaptan talpionupa uskodva ? pothinus  mögulant. caesar.  hogy azt fele. a hulyányő  apollod

------ Epoch  1
539/539 [==============================] - 6s 11ms/step - loss: 1.5418
A kiindulási mondatunk:  berrel jól eltelnék az időd az ebéd után ! észrevetted  hogy
A generált szöveg 0.1 temperature-rel:  a ki legyen minden van minden van megyint meg a